Import libraries

In [443]:
import numpy as np

# Agenda

- Homework 1 review
- Scaled partial pivoting
- Gauss-Jordan
- Jacobi iterative method

# Gauss Elimination - Scaled Partial Pivoting

When applying Gauss elimination, the are a couple of pitfalls we need to consider:
- Division by zero
- Round-off error

Sometimes the order in which the equations are presented have a profound effect on the results. Consider the following augmented coefficient matrix whose solution is $x_1 = x_2 = x_3 = 1$

$$\left[
  \begin{matrix}
      2  & -1 & 0  \\
      -1 & 2  & -1 \\
      0  & -1 & 1  \\
  \end{matrix}
\left|
  \,
  \begin{matrix}
    1    \\
    0    \\
    0    \\
  \end{matrix}
\right.
\right]
$$

If we swap the first and last row, we can see the issue with solving this system using Gaussian elimination: the first scaling factor is zero.

$$\left[
  \begin{matrix}
      0  & -1 & 1  \\
      -1 & 2  & -1 \\
      2  & -1 & 0  \\
  \end{matrix}
\left|
  \,
  \begin{matrix}
    0    \\
    0    \\
    1    \\
  \end{matrix}
\right.
\right]
$$

This kind of situation can lead to numerical errors, even if the first index isn't zero. Let's see what happens in this case if we replace zero with an extremely small number, $\varepsilon$. Then let's compute a round of elimination.

$$\left[
  \begin{matrix}
      \varepsilon  & -1 & 1  \\
      -1           & 2  & -1 \\
      2            & -1 & 0  \\
  \end{matrix}
\left|
  \,
  \begin{matrix}
    0    \\
    0    \\
    1    \\
  \end{matrix}
\right.
\right]

\implies

\left[
  \begin{matrix}
      \varepsilon  & -1                 & 1                  \\
      0            & 2 - 1/\varepsilon  & -1 + 1/\varepsilon \\
      0            & -1 + 2/\varepsilon & -2/\varepsilon     \\
  \end{matrix}
\left|
  \,
  \begin{matrix}
    0    \\
    0    \\
    1    \\
  \end{matrix}
\right.
\right]
$$

Since the computer works with a fixed word length, all numbers are rounded off to a finite number of significant figures. If $\varepsilon$ is very small, then $1/\varepsilon$ is huge and will dominate smaller numbers like in our matrix here. For sufficiently small values of $\varepsilon$, the system of equations at this step is stored as

$$\left[
  \begin{matrix}
      \varepsilon  & -1            & 1              \\
      0            & -1/\varepsilon & 1/\varepsilon  \\
      0            & 2/\varepsilon & -2/\varepsilon \\
  \end{matrix}
\left|
  \,
  \begin{matrix}
    0    \\
    0    \\
    1    \\
  \end{matrix}
\right.
\right]$$

Here we can immendiately see that this matrix, which originally had a unique solution, is now singular and cannot be solved.

In [444]:
# Gauss elimination
def gaussElim(M,v):
    n = v.size

    A = np.array(M,dtype=float)
    b = np.array(v,dtype=float)
    x = np.zeros(n)

    # Elimination Step
    for col in range(0,n-1):
        for row in range(col+1,n):

            if A[col,col] == 0.0:
                print('ERROR: divide by zero')
                break

            m = A[row,col]/A[col,col]
            A[row,:] = A[row,:]-m*A[col,:]
            b[row] = b[row]-m*b[col]

    # Back substitution
    x[n-1] = b[n-1]/A[n-1,n-1]
    for row in range(n-2,-1,-1):

        sum = 0
        for col in range(row+1,n):
            sum = sum + A[row,col]*x[col]

        x[row] = (b[row] - sum) / A[row,row]

    return x

Modify the Gauss elimination function to handle scaled partial pivoting

In [445]:
# Gauss elimination with scaled partial pivoting 
def gaussElimPivotScale(M,v):
    n = v.size

    A = np.array(M,dtype=float)
    b = np.array(v,dtype=float)
    x = np.zeros(n)

    ###############
    # ENTER CODE HERE
    ###############

    # Elimination Step
    for col in range(0,n-1):
        # eliminate entries below the pivot
        for row in range(col+1,n):
            if A[col,col] == 0.0:
                print('ERROR: divide by zero')
                break

            m = A[row,col]/A[col,col]
            A[row,:] = A[row,:]-m*A[col,:]
            b[row] = b[row]-m*b[col]

    # Back substitution
    x[n-1] = b[n-1]/A[n-1,n-1]
    for row in range(n-2,-1,-1):

        sum = 0
        for col in range(row+1,n):
            sum = sum + A[row,col]*x[col]

        x[row] = (b[row] - sum) / A[row,row]
        
    return x

Compare results and compute the residual and $L^{2}$-normed residuals

$$\varepsilon = |Ax - b|$$

$$||\varepsilon|| = ||Ax - b||$$

In [446]:
A = np.array([[    1,     -7e12,      1],
              [-3000, 2.0000023,      6],
              [ 5e10,        -1,  0.001]], dtype=float)

b = np.array([6e-3, 3e6, -2e7], dtype=float)

[6.05359686e-12 2.23517418e-07 1.23335311e+00]
1.2333531118929588 

[6.05359686e-12 0.00000000e+00 6.49292022e-04]
0.0006492920219898224


# Gauss-Jordan Elimination

Work out the following linear system of equations using the Gauss-Jordan method:

$\begin{align}
    2x + y + 2z &= 10 \nonumber  \\
    x + 2y + z &= 8 \nonumber  \\
    3x + y - 1z &= 2 \nonumber \\
\end{align}$

In [447]:
def gauss_jordan(A, b):
    n = A.shape[0]
    Ab = np.concatenate((A, b.reshape(n, 1)), axis=1)

    # iterate over each column
    for col in range(n):
        
        # partial pivoting
        pivot_row = col
        for i in range(col + 1, n):
            if abs(Ab[i][col]) > abs(Ab[pivot_row][col]):
                pivot_row = i
        Ab[[col, pivot_row]] = Ab[[pivot_row, col]]
        
        # divide the pivot row by the pivot
        pivot = Ab[col][col]
        Ab[col] /= pivot
        
        # subtract multiples of the pivot row from all other rows
        for row in range(n):
            if row == col:
                continue
            factor = Ab[row][col]
            Ab[row] -= factor * Ab[col]
    
    # extract the solution vector from the augmented matrix
    x = Ab[:, n]
    
    return x


In [448]:
A = np.array([[2, 1, 2],
              [1, 2, 1],
              [3, 1, -1]], dtype=float)

b = np.array([10, 8, 2],dtype=float)

[1. 2. 3.]


# Jacobi Iterative Method

For a linear system of equations defined by a diagonally dominant coefficient matrix, $A$, the following iterative process will converge to a solution:

$$x_{k+1} = D^{-1}(b - Ox_{k})$$

where $D$ and $O$ are the diagonal and off-diagonal matrices of $A$, respectively.

$$D = \begin{bmatrix}
a_{11} & 0 & 0 \\
0 & a_{22} & 0 \\ 
0 & 0 & a_{33} \\ 
\end{bmatrix},

 ~~~ 

O = \begin{bmatrix}
0 & a_{12} & a_{13} \\
a_{21} & 0 & a_{23} \\ 
a_{31} & a_{32} & 0 \\ 
\end{bmatrix}$$

In [449]:
def jacobi():
    return 0

In [451]:
A = np.array([[4, 1, 1],
              [3, 5, 1],
              [1, 1, 3]])

b = np.array([7, 8, 6])

[1.26086957 0.56521739 1.39130435]
[9.66045022e-11 1.60143898e-10 8.83018103e-11]


# Practice Problem

Work out the following linear system of equations by hand using the Gauss-Jordan method. Then compare the results to your code for both Gauss-Jordan and Jacobi iterative method.

$\begin{align}
    2x + 6y - 2z &= 7 \nonumber  \\
    x + 2y + 5z &= 5 \nonumber  \\
    11x - 4y + 8z &= 3 \nonumber \\
\end{align}$